In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

#data_folder = Path('/sgoinfre/goinfre/Perso/pdespres/data/')
data_folder = Path('E:/Downloads/')
#data_folder = Path('/sgoinfre/goinfre/Perso/pdespres/data/')

df = pd.read_csv(data_folder/'train2_irf_0000_corr98.csv', \
                 dtype='int8')
y = pd.read_csv(data_folder/'y2.csv', \
                sep=';', usecols=[1], dtype='int8')

print(df.shape, y.shape)

(26495, 8065) (26495, 1)


In [2]:
import pandas as pd
from keras.models import load_model
from sklearn.metrics import roc_auc_score

#best_model_14-0.778.h5 test_irf_0000_corr98
#best_model_10-0.763.h5 train2_irf_0000_corr98
file = 'E:\\Downloads\\best_model_07-0.763.h5'
test = pd.read_csv('E:\\Downloads\\test_irf_0000_corr98.csv', dtype='int8')
model = load_model(file)
valid_preds = model.predict_proba(test, verbose=0)

#roc = roc_auc_score(y, valid_preds)
#print(roc)

In [3]:
submission = pd.DataFrame(columns={'Ids', 'TARGET'})
submission['Ids'] = [i for i in range(26500,39750)]
submission['TARGET'] = valid_preds
submission = submission[['Ids', 'TARGET']]
submission.to_csv("E:\\Downloads\\submission.csv", sep=';', index=False, header={'Ids', 'TARGET'})

In [3]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, GaussianNoise
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=.3, stratify=y)

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 512, kernel_initializer = 'lecun_uniform', activation = 'relu', input_dim = X_train.shape[1]))
    classifier.add(Dropout(0.5))
    classifier.add(Dense(units = 256, kernel_initializer = 'lecun_uniform', kernel_regularizer=regularizers.l2(0.01), activation = 'relu'))
#    classifier.add(GaussianNoise(stddev=0.01))
    classifier.add(Dense(units = 128, kernel_initializer = 'lecun_uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'lecun_uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

av_roc = 0.
callbacks = [EarlyStopping(monitor='val_acc', patience=5, verbose=1), \
                ModelCheckpoint(filepath='E:\\Downloads\\best_model_{epoch:02d}-{val_acc:.3f}.h5', \
                monitor='val_acc', save_best_only=True)]

#classifier = KerasClassifier(build_fn = build_classifier, epochs=100, batch_size=300)
classifier = build_classifier()
classifier.fit(X_train, y_train, validation_data=(X_test, y_test), shuffle=True, \
               epochs=80, batch_size=100, callbacks=callbacks)

#valid_preds = classifier.predict_proba(X_test, verbose=0)[:, 1]
valid_preds = classifier.predict_proba(X_test, verbose=0)

roc = roc_auc_score(y_test, valid_preds)
print(roc)

#valid_preds = classifier.predict_proba(X_test, verbose=0)
#valid_preds = valid_preds[:, 1]
#roc = roc_auc_score(y_test, valid_preds)
#print("ROC:", roc)
#av_roc += roc

#print('Average ROC:', av_roc/nb_folds)

Train on 18546 samples, validate on 7949 samples
Epoch 1/80
18546/18546 [==============================] - 4s 198us/step - loss: 1.4018 - acc: 0.6116 - val_loss: 0.7815 - val_acc: 0.6985
Epoch 2/80
18546/18546 [==============================] - 3s 175us/step - loss: 0.6941 - acc: 0.7057 - val_loss: 0.6206 - val_acc: 0.7096
Epoch 3/80
18546/18546 [==============================] - 3s 176us/step - loss: 0.5827 - acc: 0.7265 - val_loss: 0.5728 - val_acc: 0.7227
Epoch 4/80
18546/18546 [==============================] - 3s 176us/step - loss: 0.5547 - acc: 0.7129 - val_loss: 0.5322 - val_acc: 0.7564
Epoch 5/80
18546/18546 [==============================] - 3s 176us/step - loss: 0.5184 - acc: 0.7302 - val_loss: 0.5286 - val_acc: 0.7410
Epoch 6/80
18546/18546 [==============================] - 3s 176us/step - loss: 0.5095 - acc: 0.7332 - val_loss: 0.5038 - val_acc: 0.7620
Epoch 7/80
18546/18546 [==============================] - 3s 177us/step - loss: 0.5095 - acc: 0.7343 - val_loss: 0.5041 - v

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
from keras.models import load_model
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from scipy import interp

#best_model_14-0.778.h5 test_irf_0000_corr98
file = 'E:\\Downloads\\best_model_07-0.763.h5'
classifier = load_model(file)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
aucs = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)

for train_index, test_index in kfold.split(df, y):
    #classifier.fit(df.iloc[train_index], y.iloc[train_index], validation_data=(df.iloc[test_index], y.iloc[test_index]), epochs=100, batch_size=180, callbacks=callbacks)
    valid_preds = classifier.predict_proba(df.iloc[test_index], verbose=0)#[:, 1]

#    results = cross_val_score(classifier, df, y, cv=kfold, scoring = 'roc_auc', fit_params={'callbacks': callbacks})
    fpr, tpr,_ = roc_curve(y.iloc[test_index], valid_preds)

    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    print(roc_auc)
    aucs.append(roc_auc)
    
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
print(mean_auc)

0.8880916926131492
0.8858377908059781
0.8823500460897802
0.8877472032112951
0.887038011296083
0.8908562912971028
0.8890624529233307
0.8965034636840435
0.8768219395240071
0.89962500898078
0.8880280136467863
